# Shenqu Algorithm: Basic Demonstration

## Symmetry-Based Calibration for High-Precision Systems

This notebook demonstrates the core functionality of the Shenqu Algorithm for quantifying and correcting symmetry mismatches in multi-dimensional systems.

---

**Theory**: The algorithm measures the **symmetry mismatch degree (𝒟)** of a system represented as a dynamic magic hypercube, then applies adaptive calibration to restore optimal symmetry constraints.

In [ ]:
# Import core functionality
import numpy as np
import matplotlib.pyplot as plt
from shenqu.core import generate_demo_data, calculate_symmetry_mismatch
from shenqu.calibration import AdaptiveCalibrator

print("🚀 Shenqu Algorithm Environment Ready")
print(f"NumPy version: {np.__version__}")

## 1. System Analysis: Quantifying Symmetry Mismatch

In [ ]:
# Generate ideal and noisy system states
ideal_system = np.ones((8, 8))
noisy_system = generate_demo_data(size=8, noise_level=0.03)

# Calculate symmetry mismatches
ideal_mismatch = calculate_symmetry_mismatch(ideal_system)
noisy_mismatch = calculate_symmetry_mismatch(noisy_system)

print("🔬 System Analysis Results:")
print(f"   Ideal System Symmetry Mismatch: {ideal_mismatch:.6f}")
print(f"   Noisy System Symmetry Mismatch: {noisy_mismatch:.6f}")
print(f"   Calibration Potential: {noisy_mismatch/ideal_mismatch:.1f}x improvement")

## 2. Visualization: System State Comparison

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Ideal system
im1 = axes[0, 0].imshow(ideal_system, cmap='viridis', interpolation='nearest')
axes[0, 0].set_title('Ideal System State\nPerfect Symmetry', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel(f'Symmetry Mismatch: {ideal_mismatch:.6f}')
plt.colorbar(im1, ax=axes[0, 0])

# Noisy system
im2 = axes[0, 1].imshow(noisy_system, cmap='viridis', interpolation='nearest')
axes[0, 1].set_title('Real System with Noise\nSymmetry Distortion', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel(f'Symmetry Mismatch: {noisy_mismatch:.6f}')
plt.colorbar(im2, ax=axes[0, 1])

# Difference visualization
difference = noisy_system - ideal_system
im3 = axes[1, 0].imshow(difference, cmap='RdBu_r', interpolation='nearest')
axes[1, 0].set_title('System Deviation from Ideal', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Deviation Magnitude')
plt.colorbar(im3, ax=axes[1, 0])

# Histogram of values
axes[1, 1].hist(noisy_system.flatten(), bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[1, 1].axvline(1.0, color='red', linestyle='--', linewidth=2, label='Ideal Value')
axes[1, 1].set_title('Value Distribution', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('System Parameter Value')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Adaptive Calibration Process

In [ ]:
# Initialize adaptive calibrator
calibrator = AdaptiveCalibrator(system_size=8)

# Perform calibration
print("🔄 Starting Adaptive Calibration...")
calibration_results = calibrator.calibrate(noisy_system, target_tolerance=0.001)

print("\n📊 Calibration Results:")
for key, value in calibration_results.items():
    if key != 'optimal_state':  # Don't print the full state matrix
        print(f"   {key.replace('_', ' ').title()}: {value}")

## 4. Calibration Progress Analysis

In [ ]:
# Extract calibration history
iterations = [record['iteration'] for record in calibrator.calibration_history]
mismatches = [record['mismatch'] for record in calibrator.calibration_history]

# Plot convergence
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.semilogy(iterations, mismatches, 'b-', linewidth=2, alpha=0.8)
plt.axhline(y=0.001, color='r', linestyle='--', label='Target Tolerance')
plt.xlabel('Iteration')
plt.ylabel('Symmetry Mismatch (log scale)')
plt.title('Calibration Convergence', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Show final calibrated state
final_state = calibration_results['optimal_state']
final_mismatch = calculate_symmetry_mismatch(final_state)

plt.imshow(final_state, cmap='viridis', interpolation='nearest')
plt.title(f'Calibrated System State\nFinal Mismatch: {final_mismatch:.6f}', fontweight='bold')
plt.colorbar()

plt.tight_layout()
plt.show()

print(f"\n🎯 Calibration Summary:")
print(f"   Initial Mismatch: {noisy_mismatch:.6f}")
print(f"   Final Mismatch: {final_mismatch:.6f}")
print(f"   Improvement Factor: {noisy_mismatch/final_mismatch:.2f}x")
print(f"   Iterations Required: {calibration_results['iterations']}")

## 5. Performance Benchmarking

In [ ]:
# Test algorithm performance across different noise levels
noise_levels = np.linspace(0.01, 0.1, 6)
improvement_ratios = []
iteration_counts = []

print("🧪 Running Performance Benchmark...")
for noise in noise_levels:
    # Generate system with specific noise level
    test_system = generate_demo_data(size=6, noise_level=noise)
    initial_mismatch = calculate_symmetry_mismatch(test_system)
    
    # Calibrate
    test_calibrator = AdaptiveCalibrator(system_size=6)
    results = test_calibrator.calibrate(test_system, target_tolerance=0.005)
    
    if results['success']:
        improvement_ratios.append(results['improvement_ratio'])
        iteration_counts.append(results['iterations'])
    else:
        improvement_ratios.append(1.0)  # No improvement
        iteration_counts.append(10000)  # Max iterations

# Plot benchmarking results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(noise_levels, improvement_ratios, 'o-', linewidth=2, markersize=8)
ax1.set_xlabel('Noise Level')
ax1.set_ylabel('Improvement Ratio')
ax1.set_title('Calibration Effectiveness vs Noise', fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.plot(noise_levels, iteration_counts, 's-', linewidth=2, markersize=8, color='orange')
ax2.set_xlabel('Noise Level')
ax2.set_ylabel('Iterations to Converge')
ax2.set_title('Convergence Speed vs Noise', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📈 Benchmarking Complete!")
print(f"   Average Improvement: {np.mean(improvement_ratios):.2f}x")
print(f"   Average Iterations: {np.mean(iteration_counts):.0f}")

## 6. Advanced Analysis: Multi-Dimensional Systems

In [ ]:
# Demonstration with higher-dimensional systems
print("🔮 Advanced: Multi-Dimensional System Analysis")

# Create 3D system (could represent multi-parameter ADC array)
def create_3d_system(size=4, noise_level=0.02):
    """Create a 3D system for advanced analysis."""
    ideal_3d = np.ones((size, size, size))
    noise = np.random.normal(0, noise_level, (size, size, size))
    return ideal_3d + noise

# Analyze 2D slices of 3D system
system_3d = create_3d_system(size=4, noise_level=0.03)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
slice_mismatches = []

for i, ax in enumerate(axes.flat):
    if i < 4:  # Show 4 slices
        slice_2d = system_3d[i, :, :]
        slice_mismatch = calculate_symmetry_mismatch(slice_2d)
        slice_mismatches.append(slice_mismatch)
        
        im = ax.imshow(slice_2d, cmap='plasma', interpolation='nearest')
        ax.set_title(f'Slice {i+1}\nMismatch: {slice_mismatch:.6f}', fontweight='bold')
        plt.colorbar(im, ax=ax)

plt.tight_layout()
plt.show()

print(f"\n📊 3D System Slice Analysis:")
print(f"   Average Slice Mismatch: {np.mean(slice_mismatches):.6f}")
print(f"   Maximum Slice Mismatch: {np.max(slice_mismatches):.6f}")
print(f"   Minimum Slice Mismatch: {np.min(slice_mismatches):.6f}")

## Conclusion

This demonstration shows how the Shenqu Algorithm can:

- **Quantify** symmetry mismatches in complex systems
- **Adaptively calibrate** systems toward optimal symmetry
- **Handle various noise conditions** with robust performance
- **Scale to multi-dimensional systems** for advanced applications

The algorithm provides a mathematical foundation for improving precision in ADC calibration, quantum system simulation, and other high-precision engineering applications.

---

*Next: Explore advanced features in `02-performance-benchmark.ipynb`*